In [2]:
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import argparse
from pathlib import Path

2025-06-30 21:44:13.709179: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751294653.797893    5112 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751294653.823647    5112 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751294654.012832    5112 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751294654.012849    5112 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1751294654.012850    5112 computation_placer.cc:177] computation placer alr

In [3]:
model = load_model("../models/lstm_cheat_detector.keras")
scaler = joblib.load("../models/scaler.pkl")

I0000 00:00:1751294656.655684    5112 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5484 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070, pci bus id: 0000:08:00.0, compute capability: 8.6


<h1>cheater</h1>

In [42]:
csv_path = "../data/demonstration/features/cheater/engineered_kyousuke-round2-kill3_kill_6041_to_6341.csv" 

In [43]:
target_len = 300
df = pd.read_csv(csv_path)

drop_cols = ["tick", "steamid", "label", "weapon_name", "weapon_type"]
df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors="ignore")

if df.shape[0] < target_len:
    pad_rows = target_len - df.shape[0]
    last_row = df.iloc[[-1]].copy()
    padding = pd.concat([last_row] * pad_rows, ignore_index=True)
    df = pd.concat([df, padding], ignore_index=True)

df = df.iloc[:target_len]

X_flat = df.values.reshape(-1, df.shape[-1])
X_scaled = scaler.transform(X_flat).reshape(1, target_len, df.shape[1])

In [44]:
prob = model.predict(X_scaled)[0][0]
prediction = int(prob > 0.5)
label = " CHEATER" if prediction == 1 else " LEGITIMATE"
print(f"\n Prediction: {label}")
print(f" Confidence: {prob:.2%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 193ms/step

 Prediction:  LEGITIMATE
 Confidence: 46.54%


<h1>legit</h1>

In [39]:
csv_path = "../data/demonstration/features/legit/engineered_ava-legit-firstkill_kill_126852_to_127152.csv" 

In [40]:
target_len = 300
df = pd.read_csv(csv_path)

drop_cols = ["tick", "steamid", "label", "weapon_name", "weapon_type"]
df = df.drop(columns=[col for col in drop_cols if col in df.columns], errors="ignore")

if df.shape[0] < target_len:
    pad_rows = target_len - df.shape[0]
    last_row = df.iloc[[-1]].copy()
    padding = pd.concat([last_row] * pad_rows, ignore_index=True)
    df = pd.concat([df, padding], ignore_index=True)

df = df.iloc[:target_len]

X_flat = df.values.reshape(-1, df.shape[-1])
X_scaled = scaler.transform(X_flat).reshape(1, target_len, df.shape[1])

In [41]:
prob = model.predict(X_scaled)[0][0]
prediction = int(prob > 0.5)
label = " CHEATER" if prediction == 1 else " LEGITIMATE"
print(f"\n Prediction: {label}")
print(f" Confidence: {prob:.2%}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step

 Prediction:  LEGITIMATE
 Confidence: 2.45%
